# 🐳 PARTIE 5 : DOCKER ET CONTAINERISATION

### 🌟 Ce que vous allez apprendre :
- **Containers** : Isolation et portabilité
- **Images** : Construction et gestion
- **Volumes** : Persistance des données
- **Networks** : Communication entre containers
- **Python** : Docker SDK pour l'automation

### 🛠️ Prérequis :
- Docker Desktop installé
- Exécutez d'abord la cellule système ci-dessous

---

In [32]:
exec(open("helpers/docker_helper.py").read())

📝 Vérifiez que 'ipywidgets' et 'requests' sont installés dans votre environnement Jupyter si vous utilisez l'interface HTML.
� Note: Docker lui-même doit être installé séparément (Docker Desktop).
🐳 Système d'aide Docker chargé (Jupyter) !
✨ Prêt pour la conteneurisation !


---

## 📚 SECTION 5.1 : DÉCOUVERTE DE DOCKER

### 🎯 Objectifs :
- Comprendre les concepts de base de Docker
- Vérifier l'installation et la configuration
- Explorer l'API Docker depuis Python

In [6]:
# 🐳 EXERCICE 5.1.1 : Vérification de Docker
# TODO: Vérifiez que Docker est installé et accessible

# Votre code ici

In [39]:
# 🐳 **AIDE 5.1.1 - Vérification Docker**\n
helper.help('5.1.1')

In [7]:
# 🐳 EXERCICE 5.1.2 : Exploration des conteneurs
# TODO: Listez tous les conteneurs Docker (actifs et arrêtés)

# Votre code ici

In [38]:
# 🐳 **AIDE 5.1.2 - Exploration des Conteneurs**
helper.help('5.1.2')

---

## 📚 SECTION 5.2 : GESTION DES CONTENEURS

### 🎯 Objectifs :
- Démarrer et arrêter des conteneurs
- Gérer les ports et la configuration
- Comprendre le cycle de vie des conteneurs

In [8]:
# 🐳 EXERCICE 5.2.1 : Démarrer un serveur web
# TODO: Téléchargez l'image nginx et démarrez un conteneur sur le port 8080

# Votre code ici

In [37]:
# 🐳 **AIDE 5.2.1 - Création Dockerfile**\n
helper.help('5.2.1')

In [9]:
# 🐳 EXERCICE 5.2.2 : Arrêter et nettoyer
# TODO: Arrêtez et supprimez le conteneur nginx précédent

# Votre code ici

In [36]:
# 🐳 **AIDE 5.2.2 - Build de l'Image**\n
helper.help('5.2.2')

---

## 📚 SECTION 5.3 : CONSTRUCTION D'IMAGES

### 🎯 Objectifs :
- Créer un Dockerfile
- Construire des images personnalisées
- Comprendre les layers et le cache

In [10]:
# 🐳 EXERCICE 5.3.1 : Créer une image personnalisée
# TODO: Créez un Dockerfile et construisez une image Python simple

# Votre code ici

In [34]:
# 🐳 **AIDE 5.3.1 - Gestion des Volumes**\n
helper.help('5.3.1')

---

## 📚 SECTION 5.4 : VOLUMES ET PERSISTANCE

### 🎯 Objectifs :
- Créer et gérer des volumes
- Comprendre la persistance des données
- Monter des volumes dans les conteneurs

In [11]:
# 🐳 EXERCICE 5.4.1 : Volumes persistants
# TODO: Créez un volume et testez la persistance des données

# Votre code ici

In [35]:
# 🐳 **AIDE 5.4.1 - Networks Docker**\n
helper.help('5.4.1')

---

## 🎊 FÉLICITATIONS !

Vous avez terminé le module Docker ! Vous maîtrisez maintenant :

### ✅ Ce que vous avez appris :
- **Conteneurs** : Isolation et exécution d'applications
- **Images** : Construction et gestion d'environnements
- **Volumes** : Persistance et partage de données
- **API Python** : Automation Docker avec le SDK

### 🚀 Prochaines étapes :
- Explorez Docker Compose pour les applications multi-conteneurs
- Découvrez les registries Docker (Docker Hub, privés)
- Apprenez l'orchestration avec Kubernetes

**🎯 Continuez vers la Partie 6 : MongoDB et NoSQL !**